In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

from pathlib import Path
import pandas as pd
import numpy as np

fdir = Path.cwd()
print(fdir)
sys.path.append(str(fdir/'../src'))
# from config import cfg

# from build_df import load_rna
# from merge_meta_files import load_crossref, load_pdx_meta
# import load_data
# from load_data import load_crossref, load_pdx_meta, load_pdx_meta2, load_pdx_meta_jc, load_rna, PDX_SAMPLE_COLS
# from tfrecords import original_tfr_names

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/vol/ml/apartin/projects/pdx-histo/nbs


In [4]:
datadir = Path("/vol/ml/apartin/projects/pdx-histo/PDX_Transfer_Learning_Classification/Processed_Data/Data_For_MultiModal_Learning")

In [35]:
# RNA
ge = pd.read_csv(datadir/"Data_Before_Normalization.txt", sep="\t")
# ge = pd.read_csv(datadir/"Data_After_Normalization.txt", sep="\t")

ge = ge.T
ge.columns = ge.iloc[0, :]
ge = ge.reset_index().rename(columns={"index": "Sample"})
ge = ge[1:]
ge = ge.sort_values("Sample").reset_index(drop=True)

print(ge.shape)
display(ge[:2])

Genes,Sample,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1,NCIPDM.114434~197-R~A38WG0JH1,3.19733,0.0171203,6.7915,0.0171203,0.0171203,0.0171203,0.0171203,6.26616,3.962,...,4.90197,5.19388,2.25205,1.57596,2.46228,0.0171203,4.71788,5.25872,3.61748,4.75559
2,NCIPDM.114434~197-R~A38WG3J91,2.3888,0.00742409,4.21244,0.00742409,0.00742409,0.173763,0.0227397,5.91392,3.91166,...,3.96381,4.66702,2.70755,2.59566,3.40756,0.0280666,4.72659,6.02591,5.15667,4.70261


In [59]:
# Response
rsp = pd.read_csv(datadir/"Drug_Pair_Response.txt", sep="\t")
print(rsp.shape)

rsp = rsp.sort_values("Sample").reset_index(drop=True)
rsp["single"] = [True if a == b else False for a, b in zip(rsp.Drug1, rsp.Drug2)]
rsp = rsp[~rsp["single"]].reset_index(drop=True)
# rsp = rsp[rsp.Drug2.isna()].reset_index(drop=True)
rsp = rsp.drop(columns=['Drug2'])
rsp = rsp.drop_duplicates()

rsp_sum = pd.read_csv(datadir/"Drug_Response_Data_Summary.txt", sep="\t")

print(rsp.shape)
display(rsp[:2])
rsp_sum

(6962, 6)
(4406, 6)


,Sample,Image_ID,Drug1,Response,Group,single
0,NCIPDM.114551~080-T~M667M226C29,16142,NSC.756642,0,46,False
1,NCIPDM.114551~080-T~M667M226C29,16142,NSC.754352,0,24,False


,Treatment,Response,Non-response
0,Single-drug,12,494
1,Drug-pair,30,423


In [57]:
rsp

,Sample,Image_ID,Drug1,Drug2,Response,Group,single
0,NCIPDM.114434~197-R~A35YC3,27166,NSC.125973,NSC.125973,0,0,True
1,NCIPDM.114434~197-R~A36YC9,25127,NSC.125973,NSC.125973,0,0,True
2,NCIPDM.114434~197-R~A38WG0JH1,29249,NSC.125973,NSC.125973,0,0,True
3,NCIPDM.114434~197-R~A38WG3J91,29250,NSC.125973,NSC.125973,0,0,True
4,NCIPDM.114434~197-R~A38WG5,26591,NSC.125973,NSC.125973,0,0,True
...,...,...,...,...,...,...,...
6957,NCIPDM.172845~121-B~M770J02,16049,NSC.125973,NSC.125973,0,958,True
6958,NCIPDM.172845~121-B~M770J02J24JA5,16179,NSC.125973,NSC.125973,0,958,True
6959,NCIPDM.172845~121-B~M770J02J32,16081,NSC.125973,NSC.125973,0,958,True
6960,NCIPDM.172845~121-B~M770J03J74,18227,NSC.125973,NSC.125973,0,958,True


In [44]:
# Descriptors
dd = pd.read_csv(datadir/"Standardized_Filtered_Drug_Descriptor.txt", sep="\t")

print(dd.shape)
display(dd[:2])

(51, 1994)


,Drug,MW,AMW,Sp,Si,Mp,Mi,GD,nAT,nSK,...,F10[Cl-B],Uc,Ui,Hy,TPSA(NO),TPSA(Tot),SAacc,SAdon,Vx,VvdwMG
0,NSC.119875,-1.016132,6.391122,-1.784666,-1.817551,5.860684,2.185096,5.358700,-1.825539,-2.167443,...,-0.140028,-4.216219,-4.607371,2.960688,-1.428681,-1.430001,-2.164965,-1.338042,-1.690870,-1.690864
1,NSC.123127,0.656604,-0.146676,0.414969,0.431454,-0.453737,-0.496263,-0.536775,0.442488,0.621948,...,-0.140028,0.080286,0.088246,2.277980,2.417990,2.180521,2.632592,3.292942,0.486755,0.486760


In [47]:
# Descriptors
rna = pd.read_csv(datadir/"Standardized_Normalized_RNA-seq_Data_lincs1000.txt", sep="\t")
# rna = pd.read_csv(datadir/"Standardized_Unnormalized_RNA-seq_Data_lincs1000.txt", sep="\t")

print(rna.shape)
display(rna[:2])

(494, 943)


,samples,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,NCIPDM.114434~197-R~A38WG0JH1,0.725539,-0.049166,0.527175,-0.295612,0.939099,1.991644,2.026727,-0.332576,1.528807,...,0.064793,1.133724,1.038655,0.824658,-0.238520,-0.146249,1.004284,1.318660,-0.067105,0.089109
1,NCIPDM.114434~197-R~A38WG3J91,1.854690,0.079387,0.671868,0.543544,1.349651,1.592091,1.856682,1.161365,1.164606,...,1.355155,1.446011,0.592178,0.729530,0.772658,1.919372,0.556639,1.402248,1.270158,0.218368


In [62]:
datadir = Path("/vol/ml/apartin/projects/pdx-histo/data/studies")

In [64]:
r0 = pd.read_csv(datadir/"pdm/ncipdm_drug_response", sep="\t")
r0.shape

(5379, 6)